In [2]:
import faiss
import pickle
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
# Load model
model = SentenceTransformer("intfloat/multilingual-e5-small")
if torch.cuda.is_available():
    model.to("cuda")
    print("CUDA ready:", torch.cuda.get_device_name(0))
else:
    print("CUDA not available")

CUDA ready: NVIDIA GeForce RTX 4050 Laptop GPU


In [6]:
# ✅ Cell 2: Load FAISS index and chunk_texts
with open("vector_database/chunk_texts.pkl", "rb") as f:
    chunk_texts = pickle.load(f)

index = faiss.read_index("vector_database/chunk_faiss.index")
print("✅ Loaded FAISS index and chunks")


def encode_query(text: str) -> np.ndarray:
    return model.encode([text], convert_to_numpy=True).astype("float32")

def retrieve_top_k_chunks(query: str, k: int = 3):
    q_emb = encode_query(query)
    D, I = index.search(q_emb, k)
    return [chunk_texts[i] for i in I[0]]

✅ Loaded FAISS index and chunks


In [ ]:
# ✅ Cell 4: Prepare LangChain LLMChain
strategy_prompt = """
คุณเป็นผู้เชี่ยวชาญเกม ‘หมากหนีบ’ และ 36 กลยุทธ์สามก๊ก

กติกาหมากหนีบ:
1. กระดาน 8×8, ผู้เล่นละ 8 ตัว  
2. เดินทีละ 1 ตัว ตามแนวตรง (ขึ้น–ลง–ซ้าย–ขวา)  
3. หนีบกิน:  
   - แบบ 1 ตัวหนีบสอง (กิน 2)  
   - แบบ 2 ตัวหนีบรอบ (กินตัวกลาง)  

INPUT:
{moves}

งานของคุณ:วิเคราะห์ว่าการเดินหมากนั้นมีกลยุทธ์อะไรบ้าง

OUTPUT:
ชื่อกลยุทธ์ : [ชื่อกลยุทธ์]
สาเหตุที่เลือกกลยุทธ์นั้น: [สาเหตุที่เลือกกลยุทธ์นั้น]
"""

prompt = PromptTemplate(
    input_variables=["context", "moves"],
    template=strategy_prompt.strip()
)
llm = Ollama(model="36strategies_model", temperature=0.7)
chain = LLMChain(llm=llm, prompt=prompt)


In [15]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# สร้าง prompt template สำหรับ QA
template = """
คุณเป็นผู้เชี่ยวชาญในกลยุทธ์สามก๊กและหมากหนีบ
กรุณาตอบคำถามต่อไปนี้อย่างกระชับ ชัดเจน และอิงตามความรู้:

คำถาม:
{question}

คำตอบ:
"""

# สร้าง PromptTemplate
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

# โหลดโมเดลจาก Ollama
llm = Ollama(model="36strategies_model", temperature=0.7)

# สร้าง LLMChain
qa_chain = LLMChain(llm=llm, prompt=prompt)

# ใช้งานตัวอย่าง
question = "กลยุทธ์ใดที่เหมาะกับการล้อมศัตรูโดยไม่ให้หนีได้?"
answer = qa_chain.run(question=question)

print("คำตอบ:", answer)


C:\Users\User\AppData\Local\Temp\ipykernel_13488\2292916157.py:30: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(question=question)


KeyboardInterrupt: 